## code that is causing problems

In [8]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from glob import glob
import seaborn as sns
import xesmf as xe
import pandas as pd
#import def_homebrew as hb ## homemade functions xox
from scipy.special import gamma
import netCDF4 as nc
from datetime import datetime, timedelta

import h5py ####
from PIL import Image
#from IPython.display import display #
#import cv2 # not working
import os

from scipy.ndimage import convolve, label
from skimage.measure import regionprops, find_contours
from scipy.spatial import ConvexHull, distance_matrix
from skimage.morphology import remove_small_holes ## remove holes <3
from scipy.ndimage import binary_fill_holes
from skimage import measure
from cv2 import cvtColor, COLOR_BGR2GRAY, threshold, THRESH_BINARY, THRESH_OTSU

In [9]:
## files location
ds_loc = '/home/users/esree/data/2ds/'
hvps_loc = '/home/users/esree/data/hvps/'

#file of interest
file_name = 'Export_base220730153000.h5' # example file
f2ds = h5py.File(ds_loc+ file_name,'r') # open file

# break file into two - data + time
ds_image = f2ds['ImageData'] 
ds_time = f2ds['ImageTimes']
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
##### make xarray of useful time data #####
sec_since = ds_time[:,0]
pixel_slice = ds_time[:,1]
pix_sum = pixel_slice.cumsum(dtype = 'int')

## make useful datetime format (not seconds since midnight)
# using the file name for reference
date_str = file_name[11:17]
starting_date = datetime.strptime(date_str, '%y%m%d')
time_deltas = [timedelta(seconds=float(sec)) for sec in sec_since]
utc_time = [starting_date + delta for delta in time_deltas]

time_xr =xr.Dataset({
    'utc_time':utc_time,
    'pixel_slice': pixel_slice,
    'pix_sum': pix_sum})
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#diff = np.diff(time_xr['pix_sum'][:].values) # this is finding the difference between the elements of pix_sum
#selected_values = time_xr['pix_sum'][:-1][diff > 4] # this is selecting the adjacent files in which have pixels > 4 length
# i.e. selected values contains the index of where images are > 4 in x axis


def stats_description(bw_crystal):
    '''take binary image, fill in small holes and returns object containing stats about crystal'''
    
    filled_particle = remove_small_holes(bw_crystal.image, area_threshold=4) # fill in voids within binary image - better estimation of stats # may need to be altered
    
    # can see the filled in particle if needs be
    #plt.imshow(filled_particle, cmap='gray')
    
    contours = measure.find_contours(filled_particle, 0.5)
    if contours:
        contour = max(contours, key=lambda x: x.shape[0])  # Sort contours by area (largest first) and select the largest contour
        
        labeled_image = measure.label(filled_particle)  # Label the image based on the threshold
        region = measure.regionprops(labeled_image)[0]  # Assumes largest labeled region corresponds to largest contour
        
        return region
    else:
        return None

length_threshold = 300 # mu - need this minimum length of max dimension to extract the particle
pixel_resolution = 10 # mu
desired_image_size = 120
# ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~  #
minimum_area = 10 # very quick metric to stop the processing of particles with area < 10 pixels

In [25]:
len(time_xr['pix_sum'])

300000

In [23]:
## code that is causing problems:
i = 1

one_crystal = f2ds['ImageData'][:,int(selected_values[i]):int(selected_values[i+1])] # extract 1 crystal
selected_values
#print(time_xr['utc_time'][selected_values[i]])

#ds_image

<xarray.DataArray 'pix_sum' (pix_sum: 22822)> Size: 183kB
array([     14,     225,     347, ..., 1128376, 1128890, 1131677])
Coordinates:
  * pix_sum  (pix_sum) int64 183kB 14 225 347 818 ... 1128376 1128890 1131677

In [ ]:
print(len())

In [20]:
###  set up dataframe, used to extract from raw h5 file + has stats about the particle
columns = [
    "name",
    "particle_label",
    "start_index",
    "end_index",
    "start_time",
    "end_time",
    "major_axis_length",
    "minor_axis_length",
    "orientation",
    "centroid",
    "area",
    "perimeter",
    "y0",
    "y1"
    ]
particle_df = pd.DataFrame(columns=columns)


# convert whole image true / false
binary_image = (one_crystal == 0) ## important, convert regions where 0 = True (our bits of interest), all else false
particle_data = [] # can populate with data through time

# Label the connected components - i.e. extract particles from blank space + label them
labeled_image, num_features = label(binary_image) # identify connected true areas
# labeled_image = array, with each true area given a number to identify them
# num_features = number of unique connected components in image. Have to literally have adjacent pixel, not diagonal (this will make them seperate)

props = regionprops(labeled_image) # creates quick list of properties describing each feature detected in the image.
# has: label, area, centroid, bounding box - for each feature


## only doing this if anything there i.e. if props
## want to extract only particles that surpass area threshold 
if props:
    for particle in props:
        # quickly get rid of tiny particles
        if particle.area >= minimum_area:
            ## basic info
            coords = particle.coords # basically gives coords of each point of interest
            x_values = np.unique(coords[:, 1])
            s_idx = int(selected_values[i] + x_values[0])
            e_idx = int(selected_values[i] + x_values[-1])
            #print(x_values)

            
            ## more complex stats
            spec_region = stats_description(particle)
        
            if spec_region and spec_region.major_axis_length * pixel_resolution >= length_threshold:
                # nice way of saving data - lenth + measurements are correct in microns
                one_particle_data = {
                        #"image_index": image_index,
                        "name": f'{s_idx}_{particle.label}',
                        "particle_label": particle.label,
                        "start_index": s_idx,
                        "end_index": e_idx,
                        "start_time": str(time_xr['utc_time'][s_idx].values).split('T')[1], # more friendly time
                        "end_time": str(time_xr['utc_time'][e_idx].values).split('T')[1],
                    
                        #"start_time": time_xr['utc_time'][s_idx],  # assuming 'time_xr' is pre-defined and syncs with indices
                        #"end_time": time_xr['utc_time'][e_idx],
                        "major_axis_length": spec_region.major_axis_length * pixel_resolution,
                        "minor_axis_length": spec_region.minor_axis_length * pixel_resolution,
                        "orientation": spec_region.orientation,
                        "centroid": spec_region.centroid,
                        "area": (spec_region.area * (pixel_resolution**2)),
                        "perimeter": (spec_region.perimeter * pixel_resolution),
                        "y0": coords[0][0],
                        "y1": coords[-1][0]
                        }
                
                print(time_xr['utc_time'][i])
                
                one_particle_data_df = pd.DataFrame([one_particle_data])
                particle_df = pd.concat([particle_df, one_particle_data_df], ignore_index=True)
                

In [12]:
particle_df

,name,particle_label,start_index,end_index,start_time,end_time,major_axis_length,minor_axis_length,orientation,centroid,area,perimeter,y0,y1
0,10304_41,41,10304,10353,16:55:43.208000000,16:55:43.286000000,491.870635,442.699181,0.068674,"(25.55128205128205, 21.843101343101342)",163800.0,2298.589104,18,69
